In [1]:
import pandas as pd
import jieba

import jieba.posseg as pseg
jieba.set_dictionary('jieba/dict.txt')

jieba.load_userdict('userdict/symptom.txt')
jieba.load_userdict('userdict/time.txt')
jieba.load_userdict('userdict/position.txt')
jieba.load_userdict('userdict/others.txt')
jieba.initialize()

import re
from collections import defaultdict
import matplotlib.pyplot as plt

import time as timer
start_time = timer.time()

position_replace = {
    #'疼痛':'痛','疼':'痛','痠痛':'痛','脹痛':'痛','腫痛':'痛','絞痛':'痛','悶痛':'痛',
    '腹部':'腹','肚子':'腹',
    '胸部':'胸','胸口':'胸',
    '肩胛骨':'背','背部':'背',
    '臉部':'臉','臉頰':'臉','鼻子':'臉','耳朵':'臉','唇':'臉','眼':'臉','嘴唇':'臉','額頭':'臉',
    '脖子':'頸','頸部':'頸','後頸':'頸',
    '嘴巴':'嘴','牙齦':'嘴','口腔':'嘴','唇':'嘴',
}

sym_replace = {
    '疼痛':'痛','疼':'痛','痠痛':'痛','脹痛':'痛','腫痛':'痛','絞痛':'痛','悶痛':'痛',
    '嘔吐':'吐','想吐':'吐',
    '心跳快':'心悸',
    '麻木':'麻',
    '發冷':'畏寒',
    '疹子':'紅疹',
}

kcom = pd.read_csv('Kcom_dr568410605emgcase.csv', lineterminator='\n')
kcom['Kcom'] = kcom['Kcom'].fillna("並無主訴")

print(len(kcom))
kcom.head()

Building prefix dict from /Users/kuanwen/Desktop/自然語言/project/jieba/dict.txt ...
Loading model from cache /var/folders/tv/yy9kh1h15dsfg0vcy7vbgtmw0000gn/T/jieba.u936d01f3f8b35df8fcf902039c509f73.cache
Loading model cost 1.053 seconds.
Prefix dict has been built succesfully.


241822


,Pno,Caseno,Kcom
0,15929545,"5,621",今天下午開始雙腳抽筋 現求治
1,15941825,"5,011",剛剛在家全身不適 臉潮紅 在家量血壓高 現求治
2,15965603,"6,001",據家屬訴今天早上開始食慾差 不說話 全身無力 現意識不清 119測血糖33MG/DL
3,15966387,"6,801",昨天開始頭暈不適 現未改善
4,15968549,"5,441",昨天開始左腰 左腹痛 想吐 外院檢查有結石 現持續中


In [2]:
p = pd.DataFrame(columns=['Pno','Caseno'])
kcom_out = pd.concat([pd.DataFrame(columns=['Kcom_'+str(i),'Kcom_du'+str(i)]) for i in range(1,11)], axis=1)
kcom_out = pd.concat([p, kcom_out], axis=1)

kcom_out

,Pno,Caseno,Kcom_1,Kcom_du1,Kcom_2,Kcom_du2,Kcom_3,Kcom_du3,Kcom_4,Kcom_du4,...,Kcom_6,Kcom_du6,Kcom_7,Kcom_du7,Kcom_8,Kcom_du8,Kcom_9,Kcom_du9,Kcom_10,Kcom_du10


In [3]:
max_length = 0

def replace_w(string, replace_dict):
    fin = 0
    
    while fin == 0:
        match = list()
        
        for d in replace_dict:
            if d in string:
                match.append(d)
        
        if len(match) == 0:
            fin = 1
        else:
            max_seg = ''
            for m in match:
                if len(m) > len(max_seg):
                    max_seg = m
            
            string = string.replace(max_seg, replace_dict[max_seg])
    return string
            


with open('output.txt', 'a') as the_file:

    for i in range(len(kcom)):
        split = kcom['Kcom'].loc[i].replace("有","")
        split = split.replace("會","")
        split = split.split()


        t_now = 0
        m = 0
        pre_time = list()
        event_w = list()
        event_now = list()

        for s in split: 
            t_w = list()
            sym_w = list()
            pos_w = list()

            last_t = 0
            last_n = 0
            last_f = 0
            last_sym = 0
            t = 0
            sym = 0



            line = pseg.cut(s,HMM=False)
            for k,f in line:
                print('%s %s' % (k,f), end=' ')

                if f == 't':
                    last_n = 0
                    last_sym = 0
                    last_f = 0
                    if t_now == 0:
                        if sym == 1:
                            event = dict()
                            event['time'] = t_w
                            event['sym'] = sym_w
                            event_w.append(event)
                            t_w = list()
                            sym_w = list()
                            pos_w = list()
                            sym = 0
                            t_w.append(k)
                            last_t = 1
                            t = 1  
                        elif last_t == 1 and t == 1:
                            t_w.append(k)
                            last_t = 1
                            t = 1
                        elif t == 0:
                            t_w.append(k)
                            last_t = 1
                            t = 1

                elif f == 'tnow' or f == 'tg':
                    last_n = 0
                    last_t = 0
                    last_sym = 0
                    last_f = 0
                    t_now = 1
                    if sym == 1:
                        event = dict()
                        event['time'] = t_w
                        event['sym'] = sym_w
                        event_w.append(event)
                        t_w = list()
                        sym_w = list()
                        pos_w = list()
                        sym = 0


                elif f == 'sym':
                    if last_n == 1:
                        for p in pos_w:
                            p = replace_w(p, position_replace)
                            k = replace_w(k, sym_replace)
                            sym_w.append(p+k)
                        pos_w = list()
                    else:
                        k = replace_w(k, sym_replace)
                        sym_w.append(k)

                    last_n = 0
                    last_t = 0
                    last_sym = 1
                    last_f = 0
                    sym = 1

                elif f == 'f':
                    last_f = 1
                    last_sym = 0
                    last_n = 0
                    last_t = 0
                    n = k
                elif f == 'pos':
                    if last_f == 1:
                        n = n+k
                        pos_w.append(n)
                    elif last_n == 1:
                        pos_w.append(k)
                    else: 
                        n = k
                        pos_w.append(n)

                    last_n = 1
                    last_sym = 0
                    last_f = 0
                    last_t = 0
                elif f == 'm' and last_sym == 1:
                    last_t = 0
                    last_f = 0
                    last_n = 0
                    last_sym = 0
                    if len(t_w) == 0:
                        t_w.append(k)
                        m = 1
                        last_t = 1

                else:
                    last_f = 0
                    last_t = 0
                    last_n = 0
                    last_sym = 0

            if len(sym_w) > 0:
                if t_now == 1:
                    event_now += sym_w
                else:
                    event = dict()
                    if len(t_w) == 0:
                        event['time'] = pre_time
                    else:
                        event['time'] = t_w 
                    event['sym'] = sym_w
                    event_w.append(event)
            elif len(t_w) > 0:
                    pre_time = t_w

        total = list()
        
        time = ''
        for e in event_w:
            event = dict()
            #print(e)
            if len(e['time']) > 0:
                time = ''.join(e['time'])
            event['time'] = time
            event['sym'] = e['sym']

            total.append(event)
        
        if m == 1:
            m_t = ''
            for idx, e in reversed(list(enumerate(total))):
                if len(e['time']) > 0:
                    m_t = e['time']
                else:
                    e['time'] = m_t
        
        for idx, e in reversed(list(enumerate(total))):
            if idx != 0:
                if e['time'] == total[idx-1]['time']:
                    #print(e)
                    total[idx-1]['sym'] += e['sym']
                    del total[idx]
        
        
    
        
        if len(event_now) > 0:
            total.append({'time': '現在', 'sym': event_now})
            #for e_now in event_now:
                #total.append({'time': '現在', 'sym': [e_now]})

        '''
        if len(total) > max_length:
            max_length = len(total)
        '''
        
        
        '''
        p = pd.DataFrame(columns=['Pno','Caseno'])
        temp = pd.concat([pd.DataFrame(columns=['Kcom_'+str(t),'Kcom_du'+str(t)]) for t in range(1,11)], axis=1)
        temp = pd.concat([p, temp], axis=1)
        
        temp['Pno'] = [kcom['Pno'].loc[i]]
        temp['Caseno'] = [kcom['Caseno'].loc[i]]
        
        t_idx = 1 
        for period in total:
            #print(period)
            
            if len(period['sym']) > 1:
                for s in period['sym']:
                    temp['Kcom_'+ str(t_idx)] = [s]
                    temp['Kcom_du'+ str(t_idx)] = period['time']
                    t_idx += 1
            else:
                temp['Kcom_'+ str(t_idx)] = period['sym']
                temp['Kcom_du'+ str(t_idx)] = period['time']
                t_idx += 1
        
        kcom_out = kcom_out.append(temp)
        '''
        
        print('%d. %s' % (i+1,kcom['Kcom'].loc[i]))
        #the_file.write('%d. %s\n' % (i+1,kcom['Kcom'].loc[i]))
        for period in total:
            print('時間： %s' % period['time'])
            print('症狀： %s' % ','.join(period['sym']))
            #the_file.write('時間： %s\n' % period['time'])
            #the_file.write('症狀： %s\n' % ','.join(period['sym']))
        print('\n')
        #the_file.write('\n')
    #print(max_length)
    #kcom_out.to_csv('kcom_output.csv', index=False,encoding="utf_8_sig")
    

今天下午 t 開始 v 雙腳 pos 抽筋 sym 現 tnow 求治 v 1. 今天下午開始雙腳抽筋 現求治
時間： 今天下午
症狀： 雙腳抽筋


剛剛 t 在家 r 全身不適 sym 臉潮紅 sym 在家 r 量 v 血壓高 sym 現 tnow 求治 v 2. 剛剛在家全身不適 臉潮紅 在家量血壓高 現求治
時間： 剛剛
症狀： 全身不適,臉潮紅,血壓高


據 p 家屬 n 訴 vn 今天 t 早上 t 開始 v 食慾差 sym 不說話 sym 全身 pos 無力 sym 現 tnow 意識不清 sym 119 eng 測 zg 血糖 n 33MG eng / x DL eng 3. 據家屬訴今天早上開始食慾差 不說話 全身無力 現意識不清 119測血糖33MG/DL
時間： 今天早上
症狀： 食慾差,不說話,全身無力
時間： 現在
症狀： 意識不清


昨天 t 開始 v 頭暈 sym 不適 sym 現 tnow 未 d 改善 v 4. 昨天開始頭暈不適 現未改善
時間： 昨天
症狀： 頭暈,不適


昨天 t 開始 v 左 n 腰 pos 左 n 腹 pos 痛 sym 想吐 sym 外院 n 檢查 vn 結石 n 現 tnow 持續 vd 中 n 5. 昨天開始左腰 左腹痛 想吐 外院檢查有結石 現持續中
時間： 昨天
症狀： 腹痛,吐


腹 pos 痛 sym 兩天 m 現 tnow 求治 v 6. 腹痛兩天 現求治
時間： 兩天
症狀： 腹痛


昨天晚上 t 開始 v 腹 pos 痛 sym 想吐 sym LMP eng ： x 8 eng / x 5 eng 7. 昨天晚上開始腹痛 想吐 LMP：8/5
時間： 昨天晚上
症狀： 腹痛,吐


剛剛 t 睡覺 v 忽然 d 開始 v 頭 pos 痛 sym 不適 sym 8. 剛剛睡覺忽然開始頭痛不適
時間： 剛剛
症狀： 頭痛,不適


昨天晚上 t 開始 v 腹 pos 痛 sym 不適 sym 現 tnow 求治 v 9. 昨天晚上開始腹痛不適 現求治
時間： 昨天晚上
症狀： 腹痛,不適


今天 t 早上 t 開始 v 發燒 sym 喉嚨 pos 痛 sym 咳嗽 sym 肌肉 pos 痠痛 sym 外院 n 求治 v 過 ug 現 tnow 未 d 改善 v 10. 今天早上開始

呼吸喘 sym 多天 m 現 tnow 未 d 改善 v SPO2 eng : x 94 eng % x 186. 呼吸喘多天 現未改善 SPO2:94%
時間： 多天
症狀： 呼吸喘


剛剛 t 開始 v 頭暈 sym 現 tnow 全身 pos 無力 sym 求治 v 187. 剛剛開始頭暈 現全身無力求治
時間： 剛剛
症狀： 頭暈
時間： 現在
症狀： 全身無力


今天 t 開始 v 頭暈 sym 全身 pos 無力 sym 現 tnow 求治 v 188. 今天開始頭暈 全身無力 現求治
時間： 今天
症狀： 頭暈,全身無力


今早 t 開始 v 胸悶 sym 心悸 sym 感覺 n 心臟 pos 無力 sym 喉嚨 pos 痛 sym 現 tnow 求治 v 血 n 氧 n 97 eng % x 189. 今早開始胸悶 心悸 感覺心臟無力  喉嚨痛 現求治 血氧97%
時間： 今早
症狀： 胸悶,心悸,心臟無力,喉嚨痛


今天 t 早上 t 開始 v 心悸 sym 現 tnow 求治 v SPO2 eng : x 97 eng % x 190. 今天早上開始心悸  現求治 SPO2:97%
時間： 今天早上
症狀： 心悸


剛剛 t 開始 v 喘 sym SPO2 eng : x 100 eng % x 191. 剛剛開始喘 SPO2:100%
時間： 剛剛
症狀： 喘


今天 t 晚上 t 開始 v 心悸 sym 不適 sym 現 tnow 求治 v 192. 今天晚上開始心悸不適 現求治
時間： 今天晚上
症狀： 心悸,不適


心悸 sym 不適 sym 多天 m 現 tnow 仍 zg 未 d 改善 v 193. 心悸不適多天 現仍未改善
時間： 多天
症狀： 心悸,不適


昨天 t 開始 v 發燒 sym 現 tnow 仍 zg 不適 sym 全身 pos 無力 sym 全身 pos 194. 昨天開始發燒 現仍不適 全身無力全身
時間： 昨天
症狀： 發燒
時間： 現在
症狀： 不適,全身無力


今早 t 開始 v 呼吸喘 sym 現 tnow 未 d 改善 v 血 n 氧 n 95 eng 195. 今早開始呼吸喘 現未改善 血氧95
時間： 今早
症狀： 呼吸喘


今天 t 腹 pos 痛 sym LMP eng : x

時間： 今天下午
症狀： 發燒


3天前 t 開始 v 眩暈 sym 頭暈 sym 342. 3天前開始  眩暈  頭暈
時間： 3天前
症狀： 眩暈,頭暈


早上 t 開始 v 右 n 腰 pos 痛 sym 現 tnow 求治 v 343. 早上開始右腰痛 現求治
時間： 早上
症狀： 腰痛


今天 t 晚上 t 開始 v 頻尿 sym 解 v 尿 n 疼痛 sym 現 tnow 求治 v 344. 今天晚上開始頻尿解尿疼痛 現求治
時間： 今天晚上
症狀： 頻尿,痛


今天 t 開始 v 血糖高 sym 想吐 sym 嗜睡 sym 345. 今天開始血糖高 想吐 嗜睡
時間： 今天
症狀： 血糖高,吐,嗜睡


晚上 t 開始 v 覺 v 頭暈 sym 不適 sym 346. 晚上開始覺頭暈不適
時間： 晚上
症狀： 頭暈,不適


昨天 t 開始 v 腹 pos 痛 sym 想吐 sym 腹瀉 sym 發燒 sym 現 tnow 求治 v LMP eng : x 月經 n 第三天 m 347. 昨天開始腹痛 想吐 腹瀉  發燒 現求治 LMP:月經第三天
時間： 昨天
症狀： 腹痛,吐,腹瀉,發燒


今天下午 t 開始 v 腹 pos 痛 sym 畏寒 sym 嘔吐 sym 348. 今天下午開始腹痛 畏寒 嘔吐
時間： 今天下午
症狀： 腹痛,畏寒,吐


昨天晚上 t 開始 v 血壓高 sym 現 tnow 頭 pos 痛 sym 求治 v 349. 昨天晚上開始血壓高 現頭痛求治
時間： 昨天晚上
症狀： 血壓高
時間： 現在
症狀： 頭痛


昨天 t 開始 v 情緒激動 sym 喝酒 v 350. 昨天開始情緒激動   有喝酒
時間： 昨天
症狀： 情緒激動


家屬 n 訴 vn 今天 t 安養院 nr 發現 v 胃 pos 造 v 簍 zg 管 vn 反 zg 抽 v 咖啡色 n 液 ng 351. 家屬訴 今天安養院發現胃造簍管反抽有咖啡色液


今天下午 t 開始 v 想吐 sym 左 n 腰 pos 痛 sym 血尿 sym 現 tnow 求治 v 352. 今天下午開始想吐 左腰痛 血尿 現求治
時間： 今天下午
症狀： 吐,腰痛,血尿


剛剛 t 11 eng 點 zg 發現 v 病患 n 倒 v 在 p 地上 s 現 t

時間： 剛剛
症狀： 頭暈,吐


剛開始 t 左 n 腰 pos 痛 sym 496. 剛開始左腰痛
時間： 剛開始
症狀： 腰痛


清晨 t 開始 v 上 n 腹 pos 痛 sym 厲害 a 現 tnow 求治 v 497. 清晨開始上腹痛厲害 現求治
時間： 清晨
症狀： 腹痛


三週 m 前 n 左腳 pos 開始 v 腫 sym 肢體 pos 水腫 sym 498. 三週前左腳開始 腫肢體水腫
時間： 
症狀： 腫,肢體水腫


剛 t 在 p 捷運 nz 站 v 內 v 頭暈 sym 後 n 跌倒 v 現 tnow 頭暈 sym 下巴 pos 傷口 pos 499. 剛在捷運站內頭暈後跌倒現頭暈下巴有傷口
時間： 剛
症狀： 頭暈
時間： 現在
症狀： 頭暈


據 p 119 eng 訴 vn 剛 t 路人 n 看到 v 騎 v 腳 pos 車 zg 突然 ad 暈倒 sym 現 tnow 意識 n 清醒 a 外 f on eng 頸圈 n SPO2 eng 99 eng % x 500. 據119訴 剛路人看到騎腳車突然暈倒 現意識清醒 外on 頸圈  SPO2 99%
時間： 剛
症狀： 暈倒


發燒 sym 咳嗽 sym 二天 m LMD eng 求治 v 過 ug 現 tnow 覺 v 呼吸不適 sym 喘 sym 不過 c 氣 zg SPO2 eng : x 97 eng % x 501. 發燒 咳嗽二天 LMD求治過 現覺呼吸不適 喘不過氣 SPO2:97%
時間： 二天
症狀： 發燒,咳嗽
時間： 現在
症狀： 呼吸不適,喘


星期三 t 開始 v 發燒 sym 喉嚨 pos 痛 sym 不適 sym 502. 星期三開始發燒  喉嚨痛  不適
時間： 星期三
症狀： 發燒,喉嚨痛,不適


昨晚 t 開始 v 小便 pos 灼熱感 n 頻尿 sym 右 n 腰痠 sym 血尿 sym 現 tnow 求治 v 503. 昨晚開始小便灼熱感 頻尿 右腰痠 血尿  現求治
時間： 昨晚
症狀： 頻尿,腰痠,血尿


剛開始 t 頭暈 sym 現 tnow 求治 v 504. 剛開始頭暈 現求治
時間： 剛開始
症狀： 頭暈


早上 t 開始 v 頭暈 sym 嘔吐 sym 505. 早上開始頭暈 嘔吐
時間： 早上
症狀： 頭暈,

KeyboardInterrupt: 

In [ ]:
#kcom_out.to_csv('kcom_output_0526.csv', index=False,encoding="utf_8_sig")
#kcom_out.head()

In [ ]:
#import time
print("--- %s seconds ---" % (timer.time() - start_time))

In [ ]:
#kcom_out.to_csv('kcom_output.csv', index=False,encoding="utf_8_sig")